In [ ]:
#This script takes every simulation with all points and calculates rmse and mae in each case

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('../machine-scientist/')
sys.path.append('../machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
def clean_index(dataframe):
    dataframe.set_index('Unnamed: 0', inplace=True)
    dataframe.index.name = None
    dataframe= dataframe.reset_index(drop=True)
    return dataframe

def add_bms_pred(dataframe, bms_trace, number_param, dimensions=False):

    if dimensions==True:
         VARS = ['x','y',]
         prior_par = read_prior_par('../machine-scientist/Prior/final_prior_param_sq.named_equations.nv2.np10.2016-09-09 18:49:42.600380.dat')
    else:
        VARS = ['x',]
        prior_par = read_prior_par('../machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np10.2017-10-18 18:07:35.089658.dat')

    try:
        x = dataframe[[c for c in VARS]].copy()
        y=dataframe.z
    except KeyError:
        print("HOLA!")

        
        VARS = ['x1',]
        x = dataframe[[c for c in VARS]].copy()
        y=dataframe.y

    

    #mdl model
    minrow = bms_trace[bms_trace.H == min(bms_trace.H)].iloc[0]
    minH, minexpr, minparvals = minrow.H, minrow.expr, ast.literal_eval(minrow.parvals)

    
    t = Tree(
        variables=list(x.columns),
        parameters=['a%d' % i for i in range(number_param)],
        x=x, y=y,
        prior_par=prior_par,
        max_size=200,
        from_string=minexpr,
    )

    t.set_par_values(deepcopy(minparvals))

    dplot = deepcopy(dataframe)
    dplot['zbms'] = t.predict(x)


    return dplot
    

In [4]:
#Read NN and BMS data
functions=['1', '5' , '7', '8', '10']

realizations=2
N=9
sigmas=[0.0, 0.02, 0.04,0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.20]

runid=0
NPAR=10 #10, 20
steps=50000


rmse_nn_train=[];rmse_nn_test=[]
rmse_mdl_train=[];rmse_mdl_test=[]

mae_nn_train=[];mae_nn_test=[]
mae_mdl_train=[];mae_mdl_test=[]

n_index=[];r_index=[];sigma_index=[];function_index=[]

#Put mae and rmse of each simulation (on nn and bms) in a dataframe
for function in functions:
    for sigma in sigmas:
        for r in range(realizations+1):
            
            #Save indices for dataframe
            r_index.append(r);sigma_index.append(sigma);function_index.append(function)
            
            #Read NN data: original and high resolution
            #------------------------------------------------------------------------------------------
            original_file_path='../../data/nguyen/1x_resolution/trained_nns/'
            model_d='NN_no_overfit_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            d=pd.read_csv(original_file_path + model_d)

            hr_file_path='../../data/nguyen/inter_extrapolation/'
            model_dhr='NN_no_overfit_inter_extrapolation_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            dhr=pd.read_csv(hr_file_path + model_dhr)
            #------------------------------------------------------------------------------------------

            #High resolution dataset: run over functions
            #-------------------------------------------------------------
            dnhr=dhr[dhr['rep']==int(function)]
            dnhr=clean_index(dnhr)
            #-------------------------------------------------------------

            #Read BMS data
            #-------------------------------------------------------------
            trace_path='../../data/MSTraces/nguyen/1x_resolution/'
            filename='BMS_nguyen_n_'+str(function)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_id_0_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
            trace=pd.read_csv(trace_path + filename, sep=';', header=None, names=['t', 'H', 'expr', 'parvals', 'kk1', 'kk2','kk3'])
            #-------------------------------------------------------------
            
            #Add BMS prediction to high resolution dataframe
            #-------------------------------------------------------------
            #distinguish between one variable and two-variable functions
            if function=='10':
                dinterpolate=add_bms_pred(dnhr, trace, NPAR, dimensions=True)
            else:
                dinterpolate=add_bms_pred(dnhr, trace, NPAR)
            #-------------------------------------------------------------
            
            
            #Remove points of approximation from the dataset
            #-------------------------------------------------------------
            dinterpolate=dinterpolate.round({'x1':2})
            dn=d[d['rep']==int(function)]
            dn.set_index('Unnamed: 0', inplace=True)
            dn.index.name = None
            dn=dn.reset_index(drop=True)
            dn=dn.round({'x1':2})
            
            dinterpolate=dinterpolate[-dinterpolate['x'].isin(dn['x'].tolist() )] #cross dataframes to get only high resolution points
            #-------------------------------------------------------------
            

            #Compute and save Errors to dataframe
            #-----------------------------------------------------------------------------------------------------------------------
            train_size=300
            
            rmse_nn_train_i=root_mean_squared_error(dinterpolate.loc[:train_size-1]['zmodel'],dinterpolate.loc[:train_size -1]['z'])
            rmse_nn_train.append(rmse_nn_train_i)
                
            rmse_nn_test_i=root_mean_squared_error(dinterpolate.loc[train_size-1:]['zmodel'],dinterpolate.loc[train_size -1:]['z'])
            rmse_nn_test.append(rmse_nn_test_i)

            mae_nn_train_i=mean_absolute_error(dinterpolate.loc[:train_size-1]['zmodel'],dinterpolate.loc[:train_size -1]['z'])
            mae_nn_train.append(mae_nn_train_i)
            
            mae_nn_test_i=mean_absolute_error(dinterpolate.loc[train_size-1:]['zmodel'],dinterpolate.loc[train_size -1:]['z'])
            mae_nn_test.append(mae_nn_test_i)


            try:
                rmse_mdl_train_i=root_mean_squared_error(dinterpolate.loc[:train_size-1]['zbms'],dinterpolate.loc[:train_size-1]['z'])
            except ValueError:
                rmse_mdl_train_i=np.inf
            rmse_mdl_train.append(rmse_mdl_train_i)

            try:
                rmse_mdl_test_i=root_mean_squared_error(dinterpolate.loc[train_size-1:]['zbms'],dinterpolate.loc[train_size-1:]['z'])
            except ValueError:
                rmse_mdl_test_i=np.inf
                    
            rmse_mdl_test.append(rmse_mdl_test_i)

            try:
                mae_mdl_train_i=mean_absolute_error(dinterpolate.loc[:train_size-1]['zbms'],dinterpolate.loc[:train_size -1]['z'])
            except ValueError:
                mae_mdl_train_i=np.inf
                    
            mae_mdl_train.append(mae_mdl_train_i)

            try:
                mae_mdl_test_i=mean_absolute_error(dinterpolate.loc[train_size-1:]['zbms'],dinterpolate.loc[train_size -1:]['z'])
            except ValueError:
                mae_mdl_test_i=np.inf
                
            mae_mdl_test.append(mae_mdl_test_i)
            #-----------------------------------------------------------------------------------------------------------------------
            

      
errors_df=pd.DataFrame({'sigma':sigma_index, 'function':function_index, 'mae_nn_interp.':mae_nn_train, 'mae_nn_extrap.':mae_nn_test, 
                        'mae_mdl_interp.':mae_mdl_train, 'mae_mdl_extrap.':mae_mdl_test, 'rmse_nn_interp.':rmse_nn_train, 
                        'rmse_nn_extrap.': rmse_nn_test, 'rmse_mdl_interp.':rmse_mdl_train, 'rmse_mdl_extrap.': rmse_mdl_test, 
                        'r': r_index})
errors_df.to_csv('../../data/all_errors_nguyen_interpolation' + '.csv')
display(errors_df)

,sigma,function,mae_nn_interp.,mae_nn_extrap.,mae_mdl_interp.,mae_mdl_extrap.,rmse_nn_interp.,rmse_nn_extrap.,rmse_mdl_interp.,rmse_mdl_extrap.,r
0,0.00,1,0.005506,0.807675,1.233750e-16,6.661338e-16,0.006952,1.080779,1.968682e-16,8.514391e-16,0
1,0.00,1,0.002940,0.736883,1.183226e-16,4.797260e-16,0.004161,1.023729,2.084592e-16,7.405598e-16,1
2,0.00,1,0.005565,0.913866,1.411848e-16,8.662481e-16,0.007171,1.204467,2.380712e-16,1.114327e-15,2
3,0.02,1,0.012771,0.909039,3.273081e-03,2.350146e-01,0.016572,1.189422,6.553932e-03,2.940264e-01,0
4,0.02,1,0.003716,0.768558,1.770215e-03,1.752805e-02,0.004839,1.064696,2.444490e-03,1.886998e-02,1
...,...,...,...,...,...,...,...,...,...,...,...
160,0.18,10,0.014412,0.013541,3.505487e-02,4.685644e-02,0.016481,0.015595,4.176582e-02,5.997377e-02,1
161,0.18,10,0.027221,0.042887,3.373495e-02,4.543379e-02,0.033338,0.054245,3.752217e-02,5.873141e-02,2
162,0.20,10,0.057718,0.315035,3.630886e-02,1.224666e-01,0.065818,0.338998,4.542132e-02,1.374361e-01,0
163,0.20,10,0.090682,0.163616,8.778410e-02,6.392759e-02,0.106460,0.169498,1.029743e-01,7.009051e-02,1
